In [42]:
# Author: Davide Aloi - PhD student - University of Birmingham
# Electrode coordinates plotted over MNI brain.

import numpy as np
import os
import glob
import nilearn
from nilearn import image
import matplotlib.pyplot as plt


## Parameters and variables: 
mni_path = 'C:\\Users\\davide\\Documents\\GitHub\\wp1_2_roast\\rois\\MNI152_T1_1mm_Brain.nii'
coords_folder = 'C:\\Users\\davide\\Documents\\GitHub\\wp1_2_roast\\electrode_coords\\'
mni = image.load_img(mni_path)
# Datasets names and subjects lists
db_names = ['wp2a','wp1a','wp1b']

# List of electrodes coordiantes (already in the MNI space)
wp1a_elect_anod = np.load(os.path.join(coords_folder, 'wp1a_anod_mni_coords.npy'), allow_pickle = True)
wp1a_elect_anod = np.asarray(list(dict(enumerate(wp1a_elect_anod.flatten(), 1))[1].values()))

wp2a_elect_anod = np.load(os.path.join(coords_folder, 'wp2a_anod_mni_coords.npy'), allow_pickle = True)
wp2a_elect_anod = np.asarray(list(dict(enumerate(wp2a_elect_anod.flatten(), 1))[1].values()))

wp1b_elect_cath = np.load(os.path.join(coords_folder, 'wp1b_cath_mni_coords.npy'), allow_pickle = True)
wp1b_elect_cath = np.asarray(list(dict(enumerate(wp1b_elect_cath.flatten(), 1))[1].values()))

In [43]:
# Function to convert coordinates from world to matrix space
def w_to_matrix(mni_coords, T):
    first_arg = np.transpose(np.linalg.inv(T))
    second_arg = (np.array([mni_coords[0],mni_coords[1],mni_coords[2], 1]))
    mat_coord = np.dot(second_arg,first_arg)
    mat_coord = mat_coord[0:3]
    mat_coord = np.round(mat_coord[:])
    return mat_coord
    

In [44]:
sphere = np.zeros(mni.shape)
for point in wp1a_elect_anod:
    center = w_to_matrix((point[0]), mni.affine)
    distance = np.linalg.norm(np.subtract(np.indices(mni.get_fdata().shape).T,np.asarray(center)), axis=len(center))
    sphere = np.where(distance.T < 2, 1, sphere)

for point in wp2a_elect_anod:
    center = w_to_matrix((point[0]), mni.affine)
    distance = np.linalg.norm(np.subtract(np.indices(mni.get_fdata().shape).T,np.asarray(center)), axis=len(center))
    sphere = np.where(distance.T < 2, 2, sphere)

for point in wp1b_elect_cath:
    center = w_to_matrix((point[0]), mni.affine)
    distance = np.linalg.norm(np.subtract(np.indices(mni.get_fdata().shape).T,np.asarray(center)), axis=len(center))
    sphere = np.where(distance.T < 2, 3, sphere)

In [45]:
from nilearn.image import new_img_like
# you can open this as an overlay on mricrogl to show where we extracted the data from.
new_img_like(mni, sphere).to_filename('testsphere.nii')